<a href="https://colab.research.google.com/github/lucianafem/alertaRS/blob/main/alertaRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q google-colab-selenium
!pip install -q -U google-generativeai
!pip freeze > requirements.txt

import google.generativeai as genai
import textwrap
from IPython.display import display
from IPython.display import Markdown
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sn
from prettytable import PrettyTable
import google_colab_selenium as gs
import re
import random
from random import randint
from google.colab import userdata

api_key= userdata.get('SECRET_KEY')
api_key= api_key

#GOOGLE_API_KEY=
#genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)




In [ ]:
path='/content/imersaoiadados.xlsx'
data = pd.read_excel(path)

In [ ]:
t1 = time.time()

X_train0= data.iloc[0:,1:4]
y_train0= data.iloc[0:,4:5]

scaler = StandardScaler()
X_train0 = scaler.fit_transform(X_train0)

labelencoder_classe = LabelEncoder()
y_train0 = labelencoder_classe.fit_transform(y_train0)

X_train, X_test, y_train, y_test = train_test_split(X_train0, y_train0, test_size=0.35, random_state=0) # separa o X_train para treinar
# print(len(X_train),len(X_test)) # 65 e 35. A matriz de confusão deverão somar 35 objetos.

NN = Sequential()
NN.add(Dense(8,input_dim=3, activation='tanh'))
NN.add(Dense(10, activation='tanh'))
NN.add(Dense(10, activation='tanh'))
NN.add(Dense(6, activation='softmax'))
NN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

t2 = time.time()
def accuracyscore(classificador,X_train, X_test, y_train,y_test):
    t3 = time.time()
    classificador.fit(X_train,y_train,epochs=1000,verbose=0)
    y_predictions= classificador.predict(X_test)
    y_predictions= np.argmax(y_predictions,axis=1)

    ExecTimeClassificador= round((time.time()) - t3 + (t2-t1),2)

    accuracyscore = accuracy_score(y_test, y_predictions)
    confusionmatrix=confusion_matrix(y_test, y_predictions)

    return (round(accuracyscore,2),confusionmatrix,ExecTimeClassificador)

NeuralNetworkdef= accuracyscore(NN,X_train,X_test,y_train,y_test)
NeuralNetwork_score= NeuralNetworkdef[0]
NeuralNetwork_cm= NeuralNetworkdef[1]
NeuralNetwork_time= NeuralNetworkdef[2]

print('NeuralNetwork_score=', NeuralNetwork_score)

print('NeuralNetwork_cm=', NeuralNetwork_cm)

print('Execution Time Neural Network (s) =', NeuralNetwork_time)

x = PrettyTable()
column_names = ["Machine Learning", "Score"]

x.add_column(column_names[0], ["Neural Network"])
x.add_column(column_names[1], [NeuralNetwork_score])
print(x)

x = PrettyTable()
column_names = ["Machine Learning", "Confusion Matrix"]

x.add_column(column_names[0], ["Neural Network"])
x.add_column(column_names[1], [NeuralNetwork_cm])
print(x)

x = PrettyTable()
column_names = ["Machine Learning", "Time (s)"]

x.add_column(column_names[0], ["Neural Network"])
x.add_column(column_names[1], [NeuralNetwork_time])
print(x)

cfm= NeuralNetwork_cm
classes= ["0","1", "2", "3", "4", "5"]
df_cfm= pd.DataFrame(cfm, index = classes, columns = classes)
plt.figure(figsize = (10,7))
cfm_plot= sn.heatmap(df_cfm, annot=True,fmt='.3g')  # 3g para colocar os numeros em float
cfm_plot.figure.savefig("cfm.png")

x= ["Neural Network"]

y=[NeuralNetwork_score]

plt.figure(figsize=(9,6))
plt.bar(x,y,color = '0.40', width = 0.25)  # coloca em um nivel de cinza
plt.xlabel('Machine Learning')
plt.ylabel('Accuracy')
plt.xticks(rotation=60)
for index, value in enumerate(y):
    plt.text(index-0.15,value+0.009, str(value),weight="bold")
plt.title('Accuracy -  Neural Network')
plt.show()


In [ ]:
clas= data.iloc[0:6,6:7]
rec= data.iloc[0:6,7:8]

driver = gs.Chrome()
driver.get('https://www.climatempo.com.br/previsao-do-tempo/15-dias/cidade/363/portoalegre-rs')
p= driver.find_element("xpath",'//*[@id="first-block-of-days"]/div[4]/section[1]/div[3]/div/div[3]/div[2]/div/div/span').text.strip()
p= int(re.findall(r'([\d:,.]+)',p)[0])  # procura numeros
driver.quit()
n= round(random.uniform(1, 5.3),1)
v= randint(4000, 5000)
print(f'A precipitação prevista para hoje é de {p} mm. O nível do Guaíba está em {n} m. A velocidade medida é {v} (m/s).')
X_predict= np.array([[p,n,v]])
y_predict= NN.predict(X_predict)
y_predict= np.argmax(y_predict,axis=1)

index= clas.index.get_loc(y_predict[0])
recom= rec.iloc[index]['recomendacao']
print(f'Classificação: {y_predict[0]} \n. Recomendação: {recom}.')


In [ ]:
pais= input("País: ")
estado= input("Estado: ")
cidade= input("Cidade: ")

def busca_ajuda(pais,estado,cidade):
  prompt = f'Procure no {pais} na cidade de {cidade} dentro do estado {estado} os 5 maiores hospitais que atendem \
            em caso de emergência. Mostre o endereço e o telefone de cada hospital.\
            Liste os cinco maiores pontos de abrigo.Mostre o endereço e o telefone de abrigo.\
            Liste as 5 ONGs que poderiam ajudar.Mostre o endereço e o telefone de cada ONG.'
  response = model.generate_content(prompt)
  ajuda= response.text.split("/n")
  return ajuda

ajuda=busca_ajuda(pais,estado,cidade)

for i in ajuda:
  print(i)